In [48]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
from tqdm import tqdm

file = open('./input.txt', 'r')

map = []
current_facing = (1,0)
instruction_line=False
moves_len = []
facing_dir = [current_facing]
rotations = []

def rotate(facing, rot):
    if (facing==(1,0) and rot=='R') or (facing==(-1,0) and rot=='L'):
        return (0,1)
    elif (facing==(1,0) and rot=='L') or (facing==(-1,0) and rot=='R'):
        return (0,-1)
    elif (facing==(0,1) and rot=='R') or (facing==(0,-1) and rot=='L'):
        return (-1,0)
    elif (facing==(0,1) and rot=='L') or (facing==(0,-1) and rot=='R'):
        return (1,0)
    else:
        print(facing, rot)
        raise NotImplementedError

for line in file.readlines():
    line = line.replace('\n','')
    
    if len(line)==0:
        instruction_line = True
        continue

    if not instruction_line:  

        map_line = []
        for el in line:
            if el == ' ':
                map_line.append(0)
            elif el == '.':
                map_line.append(1)
            else:
                assert el=='#', el
                map_line.append(2)
        map.append(map_line)
    else:
        assert 'R' in line, line
        last_int = ''
        for el in line:
            if el == 'R':
                current_facing = rotate(current_facing, 'R')
                moves_len.append(int(last_int))
                last_int=''
                facing_dir.append(current_facing)
                rotations.append('R')
            elif el == 'L':
                current_facing = rotate(current_facing, 'L')
                moves_len.append(int(last_int))
                last_int=''
                facing_dir.append(current_facing)
                rotations.append('L')
            else:
                last_int += el
        moves_len.append(int(last_int))


map_widths = [len(row) for row in map]
max_width = max(map_widths)
for i,row in enumerate(map):
    while len(row)<max_width:
        row.append(0)
    map[i] = row

print(f'map height: {len(map)}')
print(f'map min and max width: {min(map_widths)}, {max(map_widths)}')
print(f'num moves and directions: {len(facing_dir)}=={len(moves_len)}')
print(f'num rot: {len(rotations)}')
# print(f'moves length: {moves_len}')
# print(f'facing directions: {facing_dir}')

map height: 200
map min and max width: 50, 150
num moves and directions: 2001==2001
num rot: 2000


In [49]:
map = np.array(map).T

print(f'map shape = {map.shape}')

starting_point = (min(np.where(map[:,0]==1)[0]), 0)
print(f'starting point is {starting_point}')

map shape = (150, 200)
starting point is (50, 0)


In [50]:
def update_pos(map, pos, move, facing):

    assert map[pos] == 1, f'pos {pos} is not 1 but {map[pos]}'
    for i in range(move):
        new_pos = (pos[0]+facing[0], pos[1]+facing[1])
        if new_pos[0]<0 or new_pos[0]>=map.shape[0] or new_pos[1]<0 or new_pos[1]>=map.shape[1]:
            case = 0
        else:
            case = map[new_pos]
        if case == 1:
            pos = new_pos
        elif case == 2:
            break
        elif case == 0:
            if facing==(1,0):
                th_new_pos = (min(np.where(map[:,new_pos[1]]>0)[0]), new_pos[1])
                if map[th_new_pos]==2:
                    break
                else:
                    assert map[th_new_pos]==1
                    pos = th_new_pos
            elif facing==(-1,0):
                th_new_pos = (max(np.where(map[:,new_pos[1]]>0)[0]), new_pos[1])
                if map[th_new_pos]==2:
                    break
                else:
                    assert map[th_new_pos]==1
                    pos = th_new_pos
            elif facing==(0,1):
                th_new_pos = (new_pos[0],min(np.where(map[new_pos[0],:]>0)[0]))
                if map[th_new_pos]==2:
                    break
                else:
                    assert map[th_new_pos]==1
                    pos = th_new_pos
            elif facing==(0,-1):
                th_new_pos = (new_pos[0],max(np.where(map[new_pos[0],:]>0)[0]))
                if map[th_new_pos]==2:
                    break
                else:
                    assert map[th_new_pos]==1
                    pos = th_new_pos
            else:
                print(facing)
                raise ValueError
        else:
            print(map[new_pos])
            raise ValueError


    return pos    
        

In [51]:
current_pos = starting_point
for facing, move in zip(facing_dir, moves_len):

    current_pos = update_pos(map, current_pos, move, facing)

print(current_pos)

if facing==(1,0):
    facing_value = 0
elif facing==(0,1):
    facing_value = 1
elif facing==(-1,0):
    facing_value = 2
elif facing==(0,-1):
    facing_value = 3
print(1000*(current_pos[1]+1), 4*(current_pos[0]+1), facing_value)
print(1000*(current_pos[1]+1) + 4*(current_pos[0]+1) + facing_value)

(63, 63)
64000 256 0
64256


In [52]:
def change_face(face, facing, pos):
    if facing==(1,0):
        if face==0:
            face, facing, pos = 1,(1,0),(49-0,pos[1])
        elif face==1:
            face, facing, pos = 3,(-1,0),(49,49-pos[1])
        elif face==2:
            face, facing, pos = 1,(0,-1),(pos[1], 49)
        elif face==3:
            face, facing, pos = 1,(-1,0),(49, 49-pos[1])
        elif face==4:
            face, facing, pos = 3,(1,0),(0,pos[1])
        elif face==5:
            face, facing, pos = 3,(0,-1),(pos[1],49)
        else:
            raise ValueError            
    elif facing==(-1,0):
        if face==0:
            face, facing, pos = 4,(1,0),(0,49-pos[1])
        elif face==1:
            face, facing, pos = 0,(-1,0),(49, pos[1])
        elif face==2:
            face, facing, pos = 4,(0,1),(pos[1],0)
        elif face==3:
            face, facing, pos = 4,(-1,0),(49,pos[1])
        elif face==4:
            face, facing, pos = 0,(1,0),(0,49-pos[1])
        elif face==5:
            face, facing, pos = 0,(0,1),(pos[1],0)
        else:
            raise ValueError 
    elif facing==(0,1):
        if face==0:
            face, facing, pos = 2,(0,1),(pos[0],0)
        elif face==1:
            face, facing, pos = 2,(-1,0),(49,pos[0])
        elif face==2:
            face, facing, pos = 3,(0,1),(pos[0],0)
        elif face==3:
            face, facing, pos = 5,(-1,0),(49,pos[0])
        elif face==4:
            face, facing, pos = 5,(0,1),(pos[0],0)
        elif face==5:
            face, facing, pos = 1,(0,1),(pos[0],0)
        else:
            raise ValueError 
    elif facing==(0,-1):
        if face==0:
            face, facing, pos = 5,(1,0),(0,pos[0])  
        elif face==1:
            face, facing, pos = 5,(0,-1),(pos[0],49)
        elif face==2:
            face, facing, pos = 0,(0,-1),(pos[0],49)
        elif face==3:
            face, facing, pos = 2,(0,-1),(pos[0],49)
        elif face==4:
            face, facing, pos = 2,(1,0),(0,pos[0])
        elif face==5:
            face, facing, pos = 4,(0,-1),(pos[0],49)
        else:
            raise ValueError 
    else:
        print(facing)
        raise ValueError
    return face, facing, pos

def update_pos_cube(map_faces, face, pos, move, facing):

    assert map_faces[face][pos] == 1, f'pos {pos} is not 1 but {map_faces[face][pos]}'
    for i in range(move):
        new_pos = (pos[0]+facing[0], pos[1]+facing[1])
        if new_pos[0]<0 or new_pos[0]>=map_faces[face].shape[0] or new_pos[1]<0 or new_pos[1]>=map_faces[face].shape[1]:
            case = 0
        else:
            case = map_faces[face][new_pos]
        if case == 1:
            pos = new_pos
        elif case == 2:
            break
        elif case == 0:
            new_face, new_facing, th_new_pos = change_face(face, facing, pos)
            if map_faces[new_face][th_new_pos]==2:
                break
            else:
                assert map_faces[new_face][th_new_pos]==1
                pos = th_new_pos
                facing = new_facing
                face = new_face
        else:
            print(map[new_pos])
            raise ValueError
        # print(pos, face, facing)
    
    return pos, face, facing

In [53]:
map_faces = [None]*6
map_faces[0] = map[50:100, 0:50]
map_faces[1] = map[100:150, 0:50]
map_faces[2] = map[50:100, 50:100]
map_faces[3] = map[50:100, 100:150]
map_faces[4] = map[0:50, 100:150]
map_faces[5] = map[0:50, 150:200]

print([face.shape for face in map_faces])

starting_map_face = 0
starting_point = (0,0)


current_pos = starting_point
current_face = starting_map_face
current_facing = (1,0)

[(50, 50), (50, 50), (50, 50), (50, 50), (50, 50), (50, 50)]


In [54]:
for idx, move in enumerate(moves_len):

    current_pos, current_face, current_facing = update_pos_cube(map_faces, current_face, current_pos, move, current_facing)
    if idx<len(moves_len)-1:
        current_facing = rotate(current_facing, rotations[idx])
    # print(f'moving {move} goes to {current_pos, current_face, current_facing}')

print(current_pos)
print(current_face)
print(current_facing)

if current_facing==(1,0):
    facing_value = 0
elif current_facing==(0,1):
    facing_value = 1
elif current_facing==(-1,0):
    facing_value = 2
elif current_facing==(0,-1):
    facing_value = 3

column = (current_pos[0]+1)
row = (current_pos[1]+1)
if current_face==0:
    column+=50
elif current_face==1:
    column+=100
elif current_face==2:
    column+=50
    row+=50
elif current_face==3:
    column+=50
    row+=100
elif current_face==4:
    row+=100
elif current_face==5:
    row+=150
else:
    raise ValueError

print(column, row)

print(1000*row, 4*column, facing_value)
print(1000*row + 4*column + facing_value)


(32, 28)
2
(1, 0)
83 79
2
79000 332 0
79332
